In [1]:
import timm
import torch
import torchvision
import coremltools as ct
import numpy as np
import torch.nn as nn

In [2]:
# モデル読み込み
model_path ="/data2/celery/EfficientNetB4/cucumber_v3/2022-09-28-01-19-49/efficientnet_b4_40_model.pth"
torch_model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=10)
torch_model.load_state_dict(torch.load(model_path), strict=True)

<All keys matched successfully>

In [3]:
class TorchClassificationModel(nn.Module):
    def __init__(self):
        super(TorchClassificationModel, self).__init__()
        self.layers = nn.Sequential(
            torch_model,
            nn.Softmax(dim=1)
        )
    def forward(self, x):
        return self.layers(x)

In [4]:
torch_model = TorchClassificationModel()


In [5]:
torch_model.eval()

TorchClassificationModel(
  (layers): Sequential(
    (0): EfficientNet(
      (conv_stem): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): SiLU(inplace=True)
      (blocks): Sequential(
        (0): Sequential(
          (0): DepthwiseSeparableConv(
            (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
            (bn1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act1): SiLU(inplace=True)
            (se): SqueezeExcite(
              (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
              (act1): SiLU(inplace=True)
              (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
              (gate): Sigmoid()
            )
            (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [6]:
image_input = ct.ImageType(shape=(1, 512, 512, 3,),
                           scale=1/255)

In [11]:
random_input = torch.rand(1, 3, 512, 512)
model = torch.jit.trace(torch_model, random_input)

In [12]:
class_label = ['HEALTHY','Powdery_Mildew','Anthracnose','Downy_Mildew',
'Corynespora_Leaf_Spot','Gummy_Stem_Blight','Bacterial_Spot','CCYV','Mosaic_diseases','MYSV']

In [13]:
#Set the image scale and bias for input image preprocessing.
scale = 1.0 / (255.0 * 0.226)
red_bias = -0.485 / 0.226
green_bias = -0.456 / 0.226
blue_bias = -0.406 / 0.226

image_input = ct.ImageType(name="input_1",
                           shape=random_input.shape,
                           scale=1/255,
                        )

In [14]:
# Convert using the same API. Note that we need to provide "inputs" for pytorch conversion.
model = ct.convert(model,
                   inputs=[image_input],
                   classifier_config=ct.ClassifierConfig(class_label),
                   compute_units=ct.ComputeUnit.CPU_ONLY,
)

Running MIL Common passes:   0%|          | 0/38 [00:00<?, ? passes/s]/home/celery/.pyenv/versions/3.8.8/envs/python3.8.8/lib/python3.8/site-packages/coremltools/converters/mil/mil/passes/name_sanitization_utils.py:129: UserWarning: Output, '2111', of the source model, has been renamed to 'var_2111' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 1389/1389 [00:01<00:00, 754.11 ops/s]


In [ ]:
from PIL import Image
example_image = Image.open("/home/celery/Workspace/cucumber_circle_graphcut/sample/original_images/900216.jpg").resize((512, 512))
out_dict = model.predict({"input_1": example_image}) 

In [15]:
model.save('CucumberEffiB4.mlmodel')

In [5]:
from PIL import Image
import torch
from torchvision import models, transforms
from torch.nn import functional as F

sample_img = Image.open("/home/celery/Workspace/cucumber_circle_graphcut/sample/original_images/900216.jpg")

preprocess = transforms.Compose([
    transforms.Resize(800),
    transforms.CenterCrop(512),
    transforms.ToTensor(),
])

img_tensor = preprocess(sample_img) #[3,224,224]
input = img_tensor.unsqueeze(0) #[1,3,224,224]
output = torch_model(input)

probs = F.softmax(output, dim=1)
probs, indices = probs.sort(dim=1, descending=True)

for probs, indices in zip(probs, indices):
    for k in range(3):
        print(f"Top-{k + 1} {class_label[indices[k]]} {probs[k]:.2%}")


Top-1 MYSV 82.01%
Top-2 CCYV 2.72%
Top-3 Gummy_Stem_Blight 2.20%


In [65]:
from PIL import Image
# Load the test image and resize to 224, 224.
img_path = "/home/celery/Workspace/cucumber_circle_graphcut/sample/original_images/900216.jpg"
img = Image.open(img_path)
img = img.resize([512, 2512], Image.ANTIALIAS)

In [64]:
# Get the protobuf spec of the model.
spec = model.get_spec()
for out in spec.description.output:
    if out.type.WhichOneof('Type') == "dictionaryType":
        coreml_dict_name = out.name
        break

In [ ]:
# Make a prediction with the Core ML version of the model.
coreml_out_dict = model.predict({"input_1" : img})
print("coreml predictions: ")
print("top class label: ", coreml_out_dict["classLabel"])

coreml_prob_dict = coreml_out_dict[coreml_dict_name]

values_vector = np.array(list(coreml_prob_dict.values()))
keys_vector = list(coreml_prob_dict.keys())
top_3_indices_coreml = np.argsort(-values_vector)[:3]
for i in range(3):
    idx = top_3_indices_coreml[i]
    score_value = values_vector[idx]
    class_id = keys_vector[idx]
    print("class name: {}, raw score value: {}".format(class_id, score_value))